In [1]:
from ModelConversion import *
import os
import pm4py

import xml.etree.ElementTree as ET

In [2]:
current_directory = os.getcwd()
two_folders_up = os.path.abspath(os.path.join(current_directory, '..', '..'))
data_folder = os.path.join(two_folders_up, 'data')

In [3]:
datasets = ["BPI 2020/DomesticDeclarations", "BPI 2020/RequestForPayment", "Sepsis Cases", 
            "Traffic Fines", "Hospital Billing", "BPI 2020/InternationalDeclarations", "BPI 2020/PrepaidTravelCost",
            "BPI 2012","BPI 2020/PermitLog",  "BPI 2017"]

models = ["Alpha", "Alpha+", "IM", "IMf20", "Heuristic", "Flower"]

In [5]:
for dataset in datasets:
    for model in models:
        
        # Removes empty final markings (if they are present)
        original = open(os.path.join(data_folder, f"{dataset}/Model/{model}.pnml"), "r")
        inbetween = open(os.path.join(data_folder, f"{dataset}/Model/{model}_inbetween.pnml"), "w")
        for line in original:
            inbetween.write(line.replace("<finalmarkings/>", ""))
        original.close()
        inbetween.close()
        
        # Have pm4py auto_gues the final markings
        final = pm4py.read_pnml(os.path.join(data_folder, f"{dataset}/Model/{model}_inbetween.pnml"), auto_guess_final_marking = True)
        pm4py.write_pnml(final[0], final[1], final[2], os.path.join(data_folder, f"{dataset}/Model/{model}.pnml"))
        os.remove(os.path.join(data_folder, f"{dataset}/Model/{model}_inbetween.pnml"))
        
        # Move the finalmarkings for a new version
        tree = ET.parse(os.path.join(data_folder, f"{dataset}/Model/{model}.pnml"))
        finalmarkings = tree.getroot().find('net/finalmarkings')
        destination = tree.getroot().find('finalmarkings')
        if (finalmarkings) and (not destination):
            tree.getroot().append(finalmarkings)
            tree.getroot().find('net').remove(finalmarkings)
        
        # Save the final version
        with open(os.path.join(data_folder, f"{dataset}/Pre_Stoch_Model/{model}_.pnml"), "w") as f:
            xml_str = ET.tostring(tree.getroot(), encoding='utf-8', method='xml').decode()
            xml_with_declaration = "<?xml version='1.0' encoding='UTF-8'?>\n" + xml_str
            f.write(xml_with_declaration)
        
        #pnml_to_pnp(os.path.join(data_folder, f"{dataset}/Model/{model}.pnml"),os.path.join(data_folder, f"{dataset}/Model/{model}.pnp"))